In [ ]:
import torch
import torch.nn as nn
from src.models import EncoderTransformer, count_parameters
from src.data import random_values, create_signals, sine, add_noise, remove_samples, create_batches
from src.train import train_one_epoch, evaluate

In [ ]:
MAX_SEQUENCE_LENGTH = 512

# Hyperparameters were chosen as such to obtain a decent ratio of compute time / quality
# Dropout was set to 0 as it is basically performed by masking out parts of the input

def create_model(device):
    return EncoderTransformer(
        output_parameter_count=MAX_SEQUENCE_LENGTH,
        d_model=256,
        num_heads=16,
        num_layers=3,
        d_ff=512,
        max_seq_length=MAX_SEQUENCE_LENGTH,
        dropout=0
    ).to(device)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
transformer = create_model(device)
optim = torch.optim.Adam(transformer.parameters(), lr=0.0001)
criterion = nn.MSELoss()
count_parameters(transformer)

cuda


1712896

In [ ]:
TIME_STEP = 0.5 / MAX_SEQUENCE_LENGTH
MIN_OMEGA = 80
MAX_OMEGA = 120
HOLE_WIDTH = 50
HOLE_COUNT = 2
NOISE = 0.025
BATCH_SIZE = 128

def create_data(count: int):
    frequencies_1 = random_values(count, MIN_OMEGA, MAX_OMEGA)
    frequencies_2 = random_values(count, MIN_OMEGA, MAX_OMEGA)
    frequencies_3 = random_values(count, MIN_OMEGA, MAX_OMEGA)
    
    phases_1 = random_values(count, 0, 2 * torch.pi)
    phases_2 = random_values(count, 0, 2 * torch.pi)
    phases_3 = random_values(count, 0, 2 * torch.pi)
            
    sine_1 = create_signals(
        omegas=frequencies_1,
        signal_function=sine,
        length=MAX_SEQUENCE_LENGTH,
        time_step=TIME_STEP,
        phases=phases_1
    )

    sine_2 = create_signals(
        omegas=frequencies_2,
        signal_function=sine,
        length=MAX_SEQUENCE_LENGTH,
        time_step=TIME_STEP,
        phases=phases_2
    )

    sine_3 = create_signals(
        omegas=frequencies_3,
        signal_function=sine,
        length=MAX_SEQUENCE_LENGTH,
        time_step=TIME_STEP,
        phases=phases_3
    )

    signals = (sine_1 + sine_2 + sine_3) / 3
    messy_signals = add_noise(signals, NOISE)
    messy_signals = remove_samples(messy_signals, HOLE_WIDTH, HOLE_COUNT)
    return create_batches([messy_signals, signals], BATCH_SIZE)

In [ ]:
EPOCHS = 60
TRAIN_COUNT = 64_000
EVAL_COUNT = 6_400

eval_batched_values, eval_batched_parameters = create_data(EVAL_COUNT)

for i in range(EPOCHS):
    batched_values, batched_parameters = create_data(TRAIN_COUNT)
    train_one_epoch(transformer, optim, criterion, device, batched_values, batched_parameters)
    torch.save(transformer.state_dict(), f"{i+1}.pt")
    evaluation = evaluate(transformer, criterion, device, eval_batched_values, eval_batched_parameters)
    print(f"Epoch {i + 1}: {evaluation}")

Epoch 1 finished
Epoch 2 finished
Epoch 3 finished
Epoch 4 finished
Epoch 5 finished
Epoch 6 finished
Epoch 7 finished
Epoch 8 finished
Epoch 9 finished
Epoch 10 finished
Epoch 11 finished
Epoch 12 finished
Epoch 13 finished
Epoch 14 finished
Epoch 15 finished
Epoch 16 finished
Epoch 17 finished
Epoch 18 finished
Epoch 19 finished
Epoch 20 finished
Epoch 21 finished
Epoch 22 finished
Epoch 23 finished
Epoch 24 finished
Epoch 25 finished
Epoch 26 finished
Epoch 27 finished
Epoch 28 finished
Epoch 29 finished
Epoch 30 finished
Epoch 31 finished
Epoch 32 finished
Epoch 33 finished
Epoch 34 finished
Epoch 35 finished
Epoch 36 finished
Epoch 37 finished
Epoch 38 finished
Epoch 39 finished
Epoch 40 finished
Epoch 41 finished
Epoch 42 finished
Epoch 43 finished
Epoch 44 finished
Epoch 45 finished
Epoch 46 finished
Epoch 47 finished
Epoch 48 finished
Epoch 49 finished
Epoch 50 finished
Epoch 51 finished
Epoch 52 finished
Epoch 53 finished
Epoch 54 finished
Epoch 55 finished
Epoch 56 finished
E